> **Tip**: Welcome to the Investigate a Dataset project! You will find tips in quoted sections like this to help organize your approach to your investigation. Once you complete this project, remove these **Tip** sections from your report before submission. First things first, you might want to double-click this Markdown cell and change the title so that it reflects your dataset and investigation.

# Project: Investigate a Dataset - [No Show Appointments]

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>


## Inverstigating a dataset - No Show Appointments - kagglev2 , may/2016

### Describing the features of the data
#### This dataset collects information from 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment. A number of characteristics about the patient are included in each row.

##### ● ‘ScheduledDay’ tells us on what day the patient set up their appointment.
##### ● ‘Neighborhood’ indicates the location of the hospital.
##### ● ‘Scholarship’ indicates whether or not the patient is enrolled in Brasilian welfare program Bolsa Família.
##### ● ‘No_show’ it says ‘No’ if the patient showed up to their appointment, and ‘Yes’ if they did not show up.

##### Introductions :
##### Questions we are trying to answer :
##### 1 - what is the overall appointment show-up vs. no show-up rate
##### 2 - what are the most feature that matter the most of ('Age', 'being alcoholic','Having an SMS', 'Gender', 'Scholarship'), to make the patient make it to his appiontment?
##### importing the libraries that we gonna use in our inverstigation

In [49]:
# Use this cell to set up import statements for all of the packages that you
#   plan to use.

#import important Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

% matplotlib inline


import packages 

In [50]:
# Upgrade pandas to use dataframe.explode() function. 
!pip install --upgrade pandas==0.25.0

Requirement already up-to-date: pandas==0.25.0 in /opt/conda/lib/python3.6/site-packages (0.25.0)


<a id='wrangling'></a>
## Data Wrangling
#### checking for cleanliness, and then trim and clean the dataset for analysis.


### General Properties


In [51]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.
df=pd.read_csv('noshowappointments.csv')

In [57]:
df.head()

TypeError: __init__() got an unexpected keyword argument 'max_rows'

      PatientId  AppointmentID Gender          ScheduledDay  \
0  2.987250e+13      5642903.0      F  2016-04-29T18:38:08Z   
1  5.589978e+14      5642503.0      M  2016-04-29T16:08:27Z   
2  4.262962e+12      5642549.0      F  2016-04-29T16:19:04Z   
3  8.679512e+11      5642828.0      F  2016-04-29T17:29:31Z   
4  8.841186e+12      5642494.0      F  2016-04-29T16:07:23Z   

         AppointmentDay   Age      Neighbourhood  Scholarship  Hipertension  \
0  2016-04-29T00:00:00Z  62.0    JARDIM DA PENHA          0.0           1.0   
1  2016-04-29T00:00:00Z  56.0    JARDIM DA PENHA          0.0           0.0   
2  2016-04-29T00:00:00Z  62.0      MATA DA PRAIA          0.0           0.0   
3  2016-04-29T00:00:00Z   8.0  PONTAL DE CAMBURI          0.0           0.0   
4  2016-04-29T00:00:00Z  56.0    JARDIM DA PENHA          0.0           1.0   

   Diabetes  Alcoholism  Handcap  SMS_received No-show  
0       0.0         0.0      0.0           0.0      No  
1       0.0         0.0      0.0

to show first five rows of data

#### Checking for null values


In [ ]:
pd.isna(df).sum()
# as we can see there is no null values

In [53]:
#to get some quick information about data 
df.describe()

TypeError: __init__() got an unexpected keyword argument 'max_rows'

          PatientId  AppointmentID            Age    Scholarship  \
count  1.105270e+05   1.105270e+05  110527.000000  110527.000000   
mean   1.474963e+14   5.675305e+06      37.088874       0.098266   
std    2.560949e+14   7.129575e+04      23.110205       0.297675   
min    3.921784e+04   5.030230e+06      -1.000000       0.000000   
25%    4.172614e+12   5.640286e+06      18.000000       0.000000   
50%    3.173184e+13   5.680573e+06      37.000000       0.000000   
75%    9.439172e+13   5.725524e+06      55.000000       0.000000   
max    9.999816e+14   5.790484e+06     115.000000       1.000000   

        Hipertension       Diabetes     Alcoholism        Handcap  \
count  110527.000000  110527.000000  110527.000000  110527.000000   
mean        0.197246       0.071865       0.030400       0.022248   
std         0.397921       0.258265       0.171686       0.161543   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       

no missing values

In [6]:
df.describe()

TypeError: __init__() got an unexpected keyword argument 'max_rows'

          PatientId  AppointmentID            Age    Scholarship  \
count  1.105270e+05   1.105270e+05  110527.000000  110527.000000   
mean   1.474963e+14   5.675305e+06      37.088874       0.098266   
std    2.560949e+14   7.129575e+04      23.110205       0.297675   
min    3.921784e+04   5.030230e+06      -1.000000       0.000000   
25%    4.172614e+12   5.640286e+06      18.000000       0.000000   
50%    3.173184e+13   5.680573e+06      37.000000       0.000000   
75%    9.439172e+13   5.725524e+06      55.000000       0.000000   
max    9.999816e+14   5.790484e+06     115.000000       1.000000   

        Hipertension       Diabetes     Alcoholism        Handcap  \
count  110527.000000  110527.000000  110527.000000  110527.000000   
mean        0.197246       0.071865       0.030400       0.022248   
std         0.397921       0.258265       0.171686       0.161543   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       

to get some statistical info for our data

In [7]:
#sum duplicated values in data
df.duplicated().sum()

0

#### Looking for Data that need to be cleaned


In [54]:
# checking the info of the data (data types, null values etc..)
df.info() 

# check if there is a dplicates in the data 
print("Num of dublicated : ", + sum(df.duplicated()))

#check if there is an age with minus or 0 value
df[df["Age"] <= 0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null float64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null object
AppointmentDay    110527 non-null object
Age               110527 non-null int64
Neighbourhood     110527 non-null object
Scholarship       110527 non-null int64
Hipertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handcap           110527 non-null int64
SMS_received      110527 non-null int64
No-show           110527 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB
Num of dublicated :  0


TypeError: __init__() got an unexpected keyword argument 'max_rows'

           PatientId  AppointmentID Gender          ScheduledDay  \
59      7.184428e+13        5638545      F  2016-04-29T08:08:43Z   
63      2.366233e+14        5628286      M  2016-04-27T10:46:12Z   
64      1.885174e+14        5616082      M  2016-04-25T13:28:21Z   
65      2.718818e+14        5628321      M  2016-04-27T10:48:50Z   
67      8.647128e+13        5639264      F  2016-04-29T08:53:02Z   
89      4.314932e+14        5640380      M  2016-04-29T10:37:02Z   
101     6.822847e+13        5642813      M  2016-04-29T17:24:19Z   
104     3.326985e+12        5632495      F  2016-04-28T08:04:48Z   
132     8.468587e+14        5560742      M  2016-04-08T09:29:23Z   
150     5.392419e+10        5641476      M  2016-04-29T13:43:34Z   
188     1.883366e+14        5640933      M  2016-04-29T11:44:49Z   
192     1.165338e+12        5640561      M  2016-04-29T10:57:14Z   
193     2.379698e+11        5532452      F  2016-03-31T11:14:57Z   
194     4.442538e+11        5539566      M  2016


## Data Cleaning


### Fixing typos in data columns


In [ ]:
df.rename(columns = {'Hipertension': 'Hypertension',
                'Handcap': 'Handicap','No-show':'No_show'}, inplace = True)

### converting some columns that has date to a datetime datatype

In [ ]:
# converting some columns that has date to a datetime datatype
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])

### fixing the ages that are less then 0 by taking the mean of all ages and put those values into it


In [55]:
# we fix any age has 0 or less values because it's not make sense to have those values
# in the data (there is no patient has the exactly the age 0 or less)
meanAge = df['Age'].mean()
df[df['Age'] <= 0] = meanAge

### convert no show data to 0 and 1 instead of 'yes' and 'no' so we can handle using plot or goupby


In [56]:
df.No_show[df['No_show'] == 'Yes'] = '1'
df.No_show[df['No_show'] == 'No'] = '0'
df['No_show'] = pd.to_numeric(df['No_show'])

AttributeError: 'DataFrame' object has no attribute 'No_show'

In [ ]:
# create a mask for people who came
showed = df['No_show'] == 0
not_showed = df['No_show'] == 1
df['showed'] = showed
df['not_showed'] = not_showed

<a id='eda'></a>
## Exploratory Data Analysis

#Now that we have trimmed and cleaned data ,ready to move on exploration.



### Research Question 1 (what is the overall appointment show-up vs. no show-up rate?)
#### making a pie chart to have the percentage of who showed up and who is not


In [ ]:
allP = df['showed'].value_counts()
print(allP[1] / allP.sum() * 100)
pieChart = allP.plot.pie(figsize=(10,10), autopct='%1.1f%%', fontsize = 12);
pieChart.set_title("Status" + ' (%) (Per appointment)\n', fontsize = 15);
plt.legend();

### Research Question 2 (what are the most feature that matter the most of ('Age', 'being alcoholic','Having an SMS', 'Gender', 'Scholarship'), to make the patient make it to his appiontment?)


In [ ]:
df.head()

#### print the average age of people who show up and people who didn't

In [ ]:
df.Age[showed].mean()

In [ ]:
df.Age[not_showed].mean()

#### making an histogram to see the realtion between the (age,alcoholic people, SMS recievers, Gender, Scholarship) and the people who came to thier appointment

In [ ]:
# using group by function to find relations between features 
# visualize the average of people who came + print the percentage

df.groupby('Age')['showed'].mean().hist(alpha=0.6,bins=25,label='show');
plt.xlabel("Showed")
plt.ylabel("Age")
plt.title("Age of people who showed")
plt.legend()

In [ ]:

# using group by function to find relations between features
# and then see the relation between alcoholic people and people who showed up and who didn't
print(df.groupby('Alcoholism')['showed'].mean())
df.groupby('Alcoholism')['showed'].mean().plot(kind='bar',figsize=(22,10));
plt.xlabel("Showed")
plt.ylabel("Alcoholism")
plt.title("relation between alcoholics and patients who showed")
plt.legend()

In [ ]:
# using group by function to find relations between features
# and then see the relation between sms recievers and people who showed up and who didn't

print(df.groupby('SMS_received')['showed'].mean())
df.groupby('SMS_received')['showed'].mean().plot(kind='bar',figsize=(22,10));
plt.xlabel("Showed")
plt.ylabel("SMS_recieved")
plt.title("relation between SMS_recievers and patients who showed")
plt.legend()

In [ ]:
# using group by function to find relations between features
# and then see the relation between Gender and people who showed up and who didn't

print(df.groupby('Gender')['showed'].mean())
df.groupby('Gender')['showed'].mean().plot(kind='bar',figsize=(22,10));
plt.xlabel("Showed")
plt.ylabel("Gender")
plt.title("relation between Gender and patients who showed")
plt.legend()

#### completing the visualiztion to see what is the feature that affect patients to come.


In [ ]:
# using group by function to find relations between features
# and then see the relation between people who has scholarship and people who showed up and who didn't

print(df.groupby('Scholarship')['showed'].mean())
df.groupby('Scholarship')['showed'].mean().plot(kind = 'bar',figsize=(22,10))
plt.xlabel("Showed")
plt.ylabel("Scholarship")
plt.title("relation between patients who have scolarship and patients who showed")
plt.legend()

As we can see Age is the most factor that affect people who will attend or not!



## Limitations:
#### Missing features that could be useful to get more sure what is the most feature that impacts showing to the appointment such as if the patient is employeed or not , or whether the patient have a series medical issue or not.

#### there we some illogical data such as patients with age 0 or less


## Conclusions
#### • As we can see sending an SMS for the appiontment is not neccessary the right option to make sure that the patient will come
#### • As we can see in our investigation the Age is the most important factor that decided if a patient would come or not the average of age for people who will be most likely to show up is 39.07518726482 , and the average age for people who are not likely to show up is 35.329151291512915.
#### • As we can see about 22.8% of people that schedule an appointment did not make it to thier appointment
#### • As we can see most of people who has Scholarship are most likely to miss thier appointments with a percentage of 76.2% of showing and patients who don't have a scholarship have the percentage 80.1%
#### • the features such as different gender or alcoholic is not a factor to decide if the person would come to his appointment or not!



## Submitting your Project
#### Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).
#### Alternatively, you can download this report as .html via the File > Download as submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.
#### Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!




In [ ]:

from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])